# Solving $ A x = b $ via LU factorization and triangular solves

In this notebook, you will implement an LU factorization, solve a system with a unit lower triangular matrix, and solve a system with an upper triangular matrix.  This notebook culminates in a routine that combines these three steps into a routine that solves $ A x = b $.

<font color=red> Be sure to make a copy!!!! </font>

<h2>Preliminaries</h2>

Here is a list of laff routines that you might want to use in this notebook:
<ul>
<li> <code>laff.dots!( x, y, alpha )</code> $\alpha := x^T y + \alpha$
<li> <code>laff.invscal!( alpha, x )</code> $x := x / \alpha$ (See note below)
<li> <code>laff.axpy!( alpha, x, y )</code> $y := \alpha x + y$
<li> <code>laff.ger!( alpha, x, y, A )</code> $A := \alpha x y^T + A$
</ul>

<h2> First, let's create a matrix $ A $ and right-hand side $ b $</h2>

In [1]:
# Applying the LU Factorization to a matrix is the process of
# computing a unit lower triangular matrix, L, and upper 
# triangular matrix, U, such that A = L U.  To avoid nasty fractions
# in these caclulations, we create our matrix A from two matrices, L and U, 
# whose elements we know to be integer valued.

L = [ 1  0  0  0
     -2  1  0  0
      1 -3  1  0
      2  3 -1  1 ]

U = [ 2 -1  3 -2
      0 -2  1 -1
      0  0  1  2
      0  0  0  3 ]

A = L * U

println( "A = " )
A

A = 


4×4 Array{Int64,2}:
  2  -1   3  -2
 -4   0  -5   3
  2   5   1   3
  4  -8   8  -6

In [2]:
# create a solution vector x
x = [-1, 2, 1, -2]

4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [3]:
# store the original value of x
xold = copy(x)

# create a solution vector b so that A x = b
b = A * x
println( "b  = " )
b

b  = 


4-element Array{Int64,1}:
  3
 -7
  3
  0

In [4]:
# Much later, we are also going to solve A x = y.  Here we create that y:
x2 = [1, 2, -2, 1] 

y = A * x2
println("y = ")
y

y = 


4-element Array{Int64,1}:
  -8
   9
  13
 -34

<h2> Implement the LU factorization routine from 6.3.1 </h2>

Here is the algorithm:

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/6_3_1_1_LU.png" alt="LU factorization algorithm" width=50%>
    
<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

Create the routine
<code> LU_unb_var5!( A ) </code>
with the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html">Spark webpage</a> for the algorithm given above.

In [7]:
include("../flame.jl")
include("../laff/laff.jl")

function LU_unb_var5!(A)

    ATL, ATR, 
    ABL, ABR  = flame.part_2x2(A, 
                               0, 0, "TL")

    while size(ATL, 1) < size(A, 1)

        A00,  a01,     A02,  
        a10t, alpha11, a12t, 
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, 
                                                       ABL, ABR, 
                                                       1, 1, "BR")

        #------------------------------------------------------------#

        laff.invscal!( alpha11, a21 )        #  a21 := a21 / alpha11
        laff.ger!( -1.0, a21, a12t, A22 )    #  A22 := A22 - a21 * a12t

        #------------------------------------------------------------#

        ATL, ATR, 
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  
                                               a10t, alpha11, a12t, 
                                               A20,  a21,     A22,  
                                               "TL")

    end

    flame.merge_2x2!(ATL, ATR, 
                     ABL, ABR, A)

end

LU_unb_var5! (generic function with 1 method)

<h3> Test the routine </h3>

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

In [8]:
# recreate matrix A
A = L * U

# recreate the right-hand side
b = A * xold

# apply Gaussian elimination to matrix A
LU_unb_var5!( A )


Compare the overwritten matrix, $ A $, to the original matrices, $ L $ and $ U $.  The upper triangular part of $ A $ should equal $ U $ and the strictly lower triangular part of $ A $ should equal the strictly lower triangular part of $ L $.  

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>


In [10]:
println( "A after factorization")
A

A after factorization


4×4 Array{Int64,2}:
  2  -1   3  -2
 -2  -2   1  -1
  1  -3   1   2
  2   3  -1   3

In [11]:
println( "Original L" )
L

Original L


4×4 Array{Int64,2}:
  1   0   0  0
 -2   1   0  0
  1  -3   1  0
  2   3  -1  1

In [12]:
println( "Original U" )
U

Original U


4×4 Array{Int64,2}:
 2  -1  3  -2
 0  -2  1  -1
 0   0  1   2
 0   0  0   3

In [15]:
using LinearAlgebra
println("L and strictly lower triangular part of A are the same:")
LowerTriangular(A) - Diagonal(A) == L - Diagonal(L)

L and strictly lower triangular part of A are the same:


true

### Implement the routine `Ltrsv_unb_var1!` from 6.3.2

(if you have not yet visited Unit 6.3.2, do so now!)

Here is the algorithm:

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/6_3_2_Ltrsv.png" alt="Unit lower triangular solve algorithm" width=75%>
    
<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

Create the routine
<code> Ltrsv_unb_var1!( L, b )</code>
with the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html">Spark webpage</a> for the algorithm given above.

In [21]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Ltrsv_unb_var1!(L, b)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "TL")

    bT, 
    bB  = flame.part_2x1(b, 
                         0, "TOP")

    while size(LTL, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "BR")

        b0,    
        beta1, 
        b2     = flame.repart_2x1_to_3x1(bT, 
                                         bB, 
                                         1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( -beta1, l21, b2 ) # b2 := b2 - beta1*l21

        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "TL")

        bT, 
        bB  = flame.cont_with_3x1_to_2x1(b0,    
                                         beta1, 
                                         b2,    
                                         "TOP")

    end

    flame.merge_2x1!(bT, 
                     bB, b)

end

Ltrsv_unb_var1! (generic function with 1 method)

### Test ` Ltrsv_unb_var1!`

Take the output from <code>LU_unb_var5!</code>, and use it to solve $ L z = b $, where $ L $ is unit lower triangular with its strictly lower triangular part stored in the strictly lower triangular part of $ A $.

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

In [22]:
A

4×4 Array{Int64,2}:
  2  -1   3  -2
 -2  -2   1  -1
  1  -3   1   2
  2   3  -1   3

In [23]:
b

4-element Array{Int64,1}:
  3
 -7
  3
  0

In [24]:
Ltrsv_unb_var1!( A, b )

println( "updated b" )
b

updated b


4-element Array{Int64,1}:
  3
 -1
 -3
 -6

To be able to perform the back substitution before we implement the upper triangular solve routine described in 6.3.3, we do it the hard way here.  This allows us to see if the LU factorization followed by the solve with the unit lower triangular system gives the correct intermediate result.

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

In [25]:
x[ 4 ] = b[ 4 ] / A[ 4, 4 ]
x[ 3 ] = ( b[ 3 ] - A[ 3, 4 ] * x[ 4 ] ) / A[ 3, 3 ]
x[ 2 ] = ( b[ 2 ] - A[ 2, 3 ] * x[ 3 ] - A[ 2, 4 ] * x[ 4 ] ) / A[ 2, 2 ]
x[ 1 ] = ( b[ 1 ] - A[ 1, 2 ] * x[ 2 ] - A[ 1, 3 ] * x[ 3 ]- A[ 1, 4 ] * x[ 4 ] ) / A[ 1, 1 ]

-1.0

In [28]:
println( "x = " )
x

x = 


4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [29]:
println("x - xold = ")
x - xold

x - xold = 


4-element Array{Int64,1}:
 0
 0
 0
 0

<code> x - xold </code> should yield a zero vector

### Implement the routine `Utrsv_unb_var1!` from 6.3.3

(if you have not yet visited Unit 6.3.3, do so now!)

Here is the algorithm:

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/6_3_3_Utrsv.png" alt="Upper triangular solve algorithm" width=75%>
    
<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

Create the routine
<code>Utrsv_unb_var1!( U, b )</code>
with the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html">Spark webpage</a> for the algorithm given above.

<font color=red> Hint: Implement $ \beta_1 := \beta - u_{12}^T b_2 $ as <code> laff.dots!( -u12t, b2, beta1 ) </code> </font>

In [30]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Utrsv_unb_var1!(U, b)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "BR")

    bT, 
    bB  = flame.part_2x1(b, 
                         0, "BOTTOM")

    while size(UBR, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "TL")

        b0,    
        beta1, 
        b2     = flame.repart_2x1_to_3x1(bT, 
                                         bB, 
                                         1, "TOP")

        #------------------------------------------------------------#

        laff.dots!( -u12t, b2, beta1 ) # beta1 := beta1 - u21 * b2
        laff.invscal!( upsilon11, beta1 ) # beta1 := beta1/upsilon11

        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "BR")

        bT, 
        bB  = flame.cont_with_3x1_to_2x1(b0,    
                                         beta1, 
                                         b2,    
                                         "BOTTOM")

    end

    flame.merge_2x1!(bT, 
                     bB, b)

end

Utrsv_unb_var1! (generic function with 1 method)

### Test `Utrsv_unb_var1!`

Take the output from <code>LU_unb_var5!</code> and <code>Ltrsv_unb_var1!</code> and use it to solve $ U x = b $, where $ U $ is upper triangular and stored in the upper triangular part of $ A $.

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

In [31]:
# just to be sure, let's start over.  We'll recreate A, x, and b, run all the routines, and
# then compare the updated b to the original vector x.

A = L * U
b = A * x

LU_unb_var5!( A )
Ltrsv_unb_var1!( A, b )
Utrsv_unb_var1!( A, b )

In [32]:
println( "updated b" )
b

updated b


4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [33]:
println( "original x" )
x

original x


4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [34]:
println( "b - x" )
b - x

b - x


4-element Array{Int64,1}:
 0
 0
 0
 0

In theory, <code> b - x </code> should yield a zero vector...

### Implement the routine `Solve!` from 6.3.4

(if you have not yet visited Unit 6.3.4, do so now!)

This time, we do NOT use Spark!  What we need to do is write a routine that, when given a matrix $ A $ and right-hand side vector $ b $, solves $ A x = b $, overwriting $ A $ with the LU factorization and overwriting $ b $ with the solution vector $ x $:

<ul>
<li>
$ A \rightarrow L U $, overwriting $ A $ with $ L $ and $ U $.
</li>
<li>
Solve $ L z = b $, overwriting $ b $ with $ z $.
</li>
<li>
Solve $ U x = z $, where $ z $ is stored in vector $ b $ and $ x $ overwrites $ b $.
</li>
</ul>

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

Create the routine
<code> Solve!( A, b ) </code>

In [35]:
function Solve!( A, b )
    
    # insert appropriate calls to routines you have written here!
    LU_unb_var5!( A )
    Ltrsv_unb_var1!( A, b )
    Utrsv_unb_var1!( A, b )
end 

Solve! (generic function with 1 method)

### Test `Solve!`

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working before you rerun the one in which you are working. </font>

In [36]:
# just to be sure, let's start over.  We'll recreate A, x, and b, run all the routines, and
# then compare the updated b to the original vector x.

A = L * U
b = A * x

Solve!( A, b )

In [37]:
println("Updated b = ")
b

Updated b = 


4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [38]:
println( "original x = " )
x

original x = 


4-element Array{Int64,1}:
 -1
  2
  1
 -2

In [39]:
println( "b - x = " )
b - x

b - x = 


4-element Array{Int64,1}:
 0
 0
 0
 0

In theory, <code> b - x </code> should yield a zero vector...

<h3> What if a new right-hand side comes along? </h3>

What if we are presented with a new right-hand side, call it $ y $, with which we want to solve $ A x = y $, overwriting $ y $ with the solution?  (We created such a $ y $ at the top of this notebook.)
Notice that you can take the matrix $A $ that was modified by <code>Solve!</code> and use it with <code>Ltrsv_unb_var1!</code> and <code>Utrsv_unb_var1!</code>:

In [40]:
# insert appropriate calls here.

Ltrsv_unb_var1!( A, y )
Utrsv_unb_var1!( A, y )

In [41]:
println( "y = " )
y

y = 


4-element Array{Int64,1}:
  1
  2
 -2
  1

In [42]:
println( "x2 - y" )
x2 - y

x2 - y


4-element Array{Int64,1}:
 0
 0
 0
 0

x2 - y should evaluate to the zero vector.


<h2> <font color=red> Important: you should not refactor $ A $!!!! <font> </h2>